# Segmenting and Clustering Neighborhoods in Toronto

## Import the libraries required

The libraries required are imported.  

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install --upgrade lxml
from lxml import html # library to import data from website

!pip install --upgrade bs4 
from bs4 import BeautifulSoup # library for data parsing

!pip install geocoder
import geocoder # import geocoder

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('All libraries required are imported.')

Requirement already up-to-date: lxml in /opt/conda/envs/Python36/lib/python3.6/site-packages (4.4.2)
Requirement already up-to-date: bs4 in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.0.1)
Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

All libraries required are imported.


## Section 1: Prepare a dataframe of the neighbourhoods in Toronto

### Downloading Data and Web Scraping

Download the Wikipedia website using the requests.get method. We also check the status_code property of the Response object to make sure that the page was downloaded successfully.  

In [2]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page.status_code

200

An instance of the BeautifulSoup class is created to parse our document.  The HTML content is then printed out with appropriate formatting.  

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')

The find_all method is used to search for the table which has a class named "wikitable sortable".  The result is a long string which is then splitted by "\n" to form a list.  

In [4]:
longStr = soup.find_all('table',class_='wikitable sortable')[0].get_text()

splittedList = []
splittedList = longStr.split('\n')
splittedList[0:10]

['',
 '',
 'Postcode',
 'Borough',
 'Neighbourhood',
 '',
 '',
 'M1A',
 'Not assigned',
 'Not assigned']

From there, we processed the list and transform the data into a pandas dataframe named "neighborhoods".

In [5]:
counter = 0;
num_of_data = -1
postcode=[]
borough=[]
neighborhood=[]
for i in range(0,len(splittedList)):
    counter+=1
    if (splittedList[i] == ''):
        if(counter == 1):
            pass
        elif(counter == 2):
            pass
        else:
            print('Error')
    elif(splittedList[i] != ''):
        if (counter == 3):
            postcode.append(splittedList[i])
        elif (counter == 4):
            borough.append(splittedList[i])
        elif (counter == 5):
            neighborhood.append(splittedList[i])
            counter = 0
            num_of_data +=1
        else:
            print('Error again')
    else:
        print('Impossible')

column_names = ['PostalCode', 'borough','neighborhood']
neighborhoods = pd.DataFrame({'PostalCode':postcode,
                                      'Borough': borough,
                                      'Neighborhood': neighborhood})
neighborhoods=neighborhoods.iloc[1:]
print(neighborhoods.shape)
neighborhoods.head()



(287, 3)


,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### Preprocessing the dataframe
The rows with the borough not assigned is then dropped.  

In [6]:
neighborhoods_filtered = neighborhoods[neighborhoods["Borough"] != 'Not assigned']
neighborhoods_filtered = neighborhoods_filtered.reset_index(drop=True)
print(neighborhoods_filtered.shape)
neighborhoods_filtered.head()

(210, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


The neighborhoods dataframe is then grouped by postal code, followed by borough.  The neighborhoods that happened to fall under the same group are aggregated.  

In [7]:
neighborhoods_merged = pd.DataFrame(neighborhoods_filtered.groupby(['PostalCode','Borough'], as_index=False).agg({'Neighborhood':', '.join}))
neighborhoods_merged = neighborhoods_merged.reset_index(drop=True)
print(neighborhoods_merged.shape)
neighborhoods_merged.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Any neighborhoods with its name not assigned are given values which are the same as the borough it belongs to.  

In [8]:
finding = neighborhoods_merged['Neighborhood'].str.contains(pat='Not assigned')
for index,boolean in enumerate(finding):
    if (boolean == 0):
        pass
    elif (boolean == 1):
        assignWith = neighborhoods_merged.loc[index,'Borough']
        neighborhoods_merged['Neighborhood'].replace({'Not assigned': assignWith},inplace=True)
        print('Not assigned is replaced with borough name at row',index,'.')
        print('It was replaced with the name: ', neighborhoods_merged.loc[index,'Borough'])
    else:
        pass

Not assigned is replaced with borough name at row 85 .
It was replaced with the name:  Queen's Park


The shape of the neighborhoods dataframe.  

In [9]:
neighborhoods_merged.shape

(103, 3)

## Section 2: Prepare a dataframe containing the coordinates of each neighbourhood

### Obtain coordinates using Geocoding API
The code provided in the "my submission" section of the peer-graded assignment is modified.  The code is put into a loop to go over all postal codes.  

_The code is left in raw form as the attempt to use geocoder failed.  It took forever for the code to run, with no coordinates returned from the process._ 


### Download coordinates data 

Since the geocoder API can't be used, we resort to just download the data provided in the assignment instruction page.  Data is downloaded and transformed into a Pandas dataframe.  

In [10]:
!wget -q -O 'coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [11]:
coordinates = pd.read_csv('coordinates.csv')
print(coordinates.shape)
coordinates.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


The previous dataframe is sorted in ascending order to ease the process of merging the previous and current dataframe.  

In [12]:
neighborhoods_merged.sort_values(['PostalCode'], ascending = True, axis = 0, inplace = True)
neighborhoods_merged.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


The two dataframes are merged while making sure that the postal code of the two dataframes are identical.  Any mismatch in postal codes will result in "NaN" appearing in the latitude and longitude columns.  

In [13]:
neighborhoods_merged['Latitude'] = np.where((neighborhoods_merged['PostalCode'] == coordinates['Postal Code']), coordinates['Latitude'], np.nan)
neighborhoods_merged['Longitude'] = np.where((neighborhoods_merged['PostalCode'] == coordinates['Postal Code']), coordinates['Longitude'], np.nan)
print(neighborhoods_merged.shape)
neighborhoods_merged.head(5)

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


The following code checks if there's any mismatch in postal codes found in the two dataframes.  

In [14]:
neighborhoods_merged.isnull().sum()

PostalCode      0
Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

## Section 3: Explore and Cluster the Neighborhoods in the City of Toronto
### Visualise Toronto
The coordinates for Toronto is determined using the geocode library.  

In [15]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto is 43.653963, -79.387207.


All postal codes in the neighborhoods_merged dataframe are plotted on a Folium map created using the coordinates obtained above.  

In [16]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods_merged['Latitude'], neighborhoods_merged['Longitude'], neighborhoods_merged['Borough'], neighborhoods_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Explore each postal code using Foursquare API
Client ID, secret and version are stored as variable.  

In [17]:
# The code was removed by Watson Studio for sharing.

CLIENT_ID is stored. 
CLIENT_SECRET is stored.


A function is defined to return a dataframe containing details about the venues around the coordinates being inputed into the function.  

In [18]:
def getNearbyVenues(postalcode, latitudes, longitudes, LIMIT=100, radius=500):
    
    venues_list=[]
    for postalcode, lat, lng in zip(postalcode, latitudes, longitudes):
        print(postalcode)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postalcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

The function is used to explore all the coordinates we have in the neighborhoods_merged dataframe.  

In [19]:
toronto_venues = getNearbyVenues(postalcode=neighborhoods_merged['PostalCode'],
                                   latitudes=neighborhoods_merged['Latitude'],
                                   longitudes=neighborhoods_merged['Longitude']
                                  )

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


The result is a dataframe containing details of venues for each postal code as shown below:

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(2221, 7)


,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


### One hot encoding of venue categories
The number of venues fetched for each postal code is determined.  

In [21]:
toronto_venues.groupby('Postal Code').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M1B,1,1,1,1,1,1
M1C,1,1,1,1,1,1
M1E,7,7,7,7,7,7
M1G,3,3,3,3,3,3
M1H,8,8,8,8,8,8
M1J,2,2,2,2,2,2
M1K,7,7,7,7,7,7
M1L,10,10,10,10,10,10
M1M,2,2,2,2,2,2


The number of unique venue categories are determined too.  

In [22]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 277 uniques categories.


The dataframe "toronto_onehot" is created with each venue category becoming the columns of the dataframe.  

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(2221, 278)


,PostalCode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0

"toronto_onehot" is then grouped according to postal codes to normalise the count obtained.  

In [24]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped.head()

,PostalCode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0

### Study the venues data obtained
Using the dataframe "toronto_grouped", the top 5 venues for each postal code are determined and printed out.  

In [25]:
num_top_venues = 5

for hood in toronto_grouped['PostalCode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                             venue  freq
0             Fast Food Restaurant   1.0
1                Accessories Store   0.0
2                    Metro Station   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----M1C----
                             venue  freq
0                              Bar   1.0
1                Accessories Store   0.0
2               Mexican Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----M1E----
                 venue  freq
0         Intersection  0.14
1       Medical Center  0.14
2  Rental Car Location  0.14
3    Electronics Store  0.14
4   Mexican Restaurant  0.14


----M1G----
                             venue  freq
0                      Coffee Shop  0.67
1                Korean Restaurant  0.33
2                Accessories Store  0.00
3              Monument / Landmark  0.00
4  Molecular Gastronomy Restaurant  0.00


----M1H----
              venue 

We then create a dataframe "postalcode_venues_sorted" that store the information obtained above.  

_At here, we noticed from "postalcode_venues_sorted.shape" that there are two postal codes that didn't have any venue fetched from FourSquare.  These two postal codes will be removed at later stage._

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories.sort_values(ascending = False, axis = 0, inplace = True)
    row_categories_sorted=pd.DataFrame({'freq':row_categories})
    row_categories_sorted.reset_index(inplace=True)
    
    trigger = 0
    while(trigger < num_top_venues-1):
        # print('I am here. ')
        for index,row in row_categories_sorted.head(10).iterrows():
            # print('Start!')
            # print('index :', index)
            # print('index: ',index)
            # print('venue: ', row['index'])
            # print('freq: ', row['freq'])
            if (row['freq'] < 0.005):
                for i in range (index,num_top_venues):
                    row_categories_sorted.replace(to_replace=row_categories_sorted.iloc[i,0], value=np.NaN,inplace=True)
                    # print('Row ',i, 'has ',row_categories_sorted.iloc[i,0],'replaced with NaN.')
                break
            elif (row['freq'] > 0.005):
                trigger += 1 
        trigger = num_top_venues
        # print('End')
    # print('Let us see')
    # print(row_categories_sorted.iloc[0:num_top_venues,0].values)

    return row_categories_sorted.iloc[0:num_top_venues,0].values

In [67]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postalcode_venues_sorted = pd.DataFrame(columns=columns)
postalcode_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    postalcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

print(postalcode_venues_sorted.shape)
postalcode_venues_sorted.head()

(99, 11)


,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Bar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M1E,Rental Car Location,Pizza Place,Intersection,Mexican Restaurant,Electronics Store,Medical Center,Breakfast Spot,NaN,NaN,NaN
3,M1G,Coffee Shop,Korean Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M1H,Bakery,Hakka Restaurant,Caribbean Restaurant,Gas Station,Athletics & Sports,Thai Restaurant,Fried Chicken Joint,Bank,NaN,NaN


### Cluster the postal codes
The K-Means Clustering method is applied to the dataframe to cluster them into 5 clusters.  

In [68]:
# set number of clusters
kclusters = 12
toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)
print(toronto_grouped_clustering.shape)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

(99, 277)


array([ 3,  8,  6,  5,  6,  4,  6,  6,  6,  6,  6,  6,  6,  0,  4,  0,  6,
        6,  1, 11,  6,  2,  0,  2,  1,  6,  6,  6, 10,  2,  6,  6,  0,  0,
        6,  6,  6,  6,  2,  6,  6,  6,  2,  6,  6,  6,  4,  6,  4,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  9,  2,  6,  6,  6,  6,  6,
        6,  6,  0,  6,  2,  6,  6,  6,  6,  2,  3,  0,  6,  6,  6,  6,  6,
        6,  0,  0,  2,  6,  6,  0,  0,  1,  2,  0,  2,  0,  7],
      dtype=int32)

The cluster labels, details of postal codes and venue catgories are merged together to form a new dataframe.

_Please note that two postal codes were dropped here as no venue can be found at these postal codes._

In [69]:
# add clustering labels
postalcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
postalcode_venues_sorted.head()

toronto_merged = neighborhoods_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(postalcode_venues_sorted.set_index('PostalCode'), on='PostalCode')
toronto_merged.dropna(subset=["Cluster Labels"],axis=0, inplace=True)
toronto_merged.reset_index(inplace=True)
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype(int)

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(99, 17)


,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,3,Fast Food Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,8,Bar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,6,Rental Car Location,Pizza Place,Intersection,Mexican Restaurant,Electronics Store,Medical Center,Breakfast Spot,NaN,NaN,NaN
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917,5,Coffee Shop,Korean Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,6,Bakery,Hakka Restaurant,Caribbean Restaurant,Gas Station,Athletics & Sports,Thai Restaurant,Fried Chicken Joint,Bank,NaN,NaN


### Visualise the cluster result

A Folium map is then created to visualise how the clustering was made above.  

In [70]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the Clusters
The cluster created are studied to determine the characteristics of each cluster.

#### Cluster 1

In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,M1T,-79.304302,0,Pizza Place,Fried Chicken Joint,Fast Food Restaurant,Chinese Restaurant,Noodle House,Pharmacy,Thai Restaurant,Bank,Italian Restaurant,NaN
15,M1W,-79.318389,0,Fast Food Restaurant,Chinese Restaurant,Gym,Pizza Place,Bubble Tea Shop,Sandwich Place,Grocery Store,Coffee Shop,Nail Salon,Breakfast Spot
22,M2R,-79.442259,0,Discount Store,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Home Service,NaN,NaN,NaN,NaN
32,M4A,-79.315572,0,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,Financial or Legal Service,NaN,NaN,NaN,NaN
33,M4B,-79.309937,0,Fast Food Restaurant,Pizza Place,Intersection,Athletics & Sports,Gastropub,Café,Bank,Pharmacy,Gym / Fitness Center,NaN
70,M6B,-79.445073,0,Bakery,Pizza Place,Pub,Japanese Restaurant,NaN,NaN,NaN,NaN,NaN,NaN
79,M6N,-79.487262,0,Brewery,Grocery Store,Pizza Place,Caribbean Restaurant,Bus Line,NaN,NaN,NaN,NaN,NaN
86,M8V,-79.501321,0,Gym,Pizza Place,Fast Food Restaurant,Sandwich Place,Café,Restaurant,Fried Chicken Joint,Bakery,Liquor Store,Pharmacy
87,M8W,-79.543484,0,Pizza Place,Gym,Skating Rink,Sandwich Place,Pub,Pool,Coffee Shop,Dance Studio,Pharmacy,NaN
91,M9C,-79.577201,0,Café,Convenience Store,Liquor Store,Shopping Plaza,Pharmacy,Pizza Place,Coffee Shop,Beer Store,NaN,NaN


It is observed that this cluster share a similarities of having more than 4 venue categories.  All these areas also have pizza place(s).  

#### Cluster 2

In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,M2K,-79.385975,1,Chinese Restaurant,Bank,Café,Japanese Restaurant,NaN,NaN,NaN,NaN,NaN,NaN
24,M3B,-79.352188,1,Gym / Fitness Center,Baseball Field,Café,Japanese Restaurant,Caribbean Restaurant,NaN,NaN,NaN,NaN,NaN
93,M9M,-79.532242,1,Furniture / Home Store,Baseball Field,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


These areas have either a baseball field and/or a cafe.  All the venue categories in this cluster start with one of the first ten alphabets (A-J).  

#### Cluster 3

In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,M2P,-79.400049,2,Park,Bank,Convenience Store,Bar,NaN,NaN,NaN,NaN,NaN,NaN
23,M3A,-79.329656,2,Park,Construction & Landscaping,Food & Drink Shop,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,M3L,-79.506944,2,Bank,Park,Grocery Store,Shopping Mall,Hotel,NaN,NaN,NaN,NaN,NaN
38,M4J,-79.338106,2,Park,Rental Car Location,Convenience Store,Coffee Shop,NaN,NaN,NaN,NaN,NaN,NaN
42,M4N,-79.388790,2,Bus Line,Park,Lawyer,Swim School,NaN,NaN,NaN,NaN,NaN,NaN
62,M5P,-79.411307,2,Park,Sushi Restaurant,Jewelry Store,Trail,NaN,NaN,NaN,NaN,NaN,NaN
72,M6E,-79.453512,2,Park,Women's Store,Fast Food Restaurant,Market,NaN,NaN,NaN,NaN,NaN,NaN
77,M6L,-79.490074,2,Park,Bakery,Construction & Landscaping,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,M8X,-79.506944,2,Park,River,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,M9N,-79.518188,2,Park,Convenience Store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


All these areas (except M3L) have "park" as the 1st most common venue.   

#### Cluster 4

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,-79.194353,3,Fast Food Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,M6M,-79.476013,3,Fast Food Restaurant,Sandwich Place,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The areas in cluster 4 have only 1-2 venue categories found, with 'Fast Food Restaurant' being one of them.  

#### Cluster 5

In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M1J,-79.239476,4,Convenience Store,Playground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,M1V,-79.284577,4,Park,Playground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,M4T,-79.383160,4,Park,Playground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,M4W,-79.377529,4,Park,Trail,Playground,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Areas in clusters have only 2-3 venue categories found, with 'playground' and/or 'park' being one/two of them.  

#### Cluster 6

In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M1G,-79.216917,5,Coffee Shop,Korean Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cluster 7

In [79]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1E,-79.188711,6,Rental Car Location,Pizza Place,Intersection,Mexican Restaurant,Electronics Store,Medical Center,Breakfast Spot,NaN,NaN,NaN
4,M1H,-79.239476,6,Bakery,Hakka Restaurant,Caribbean Restaurant,Gas Station,Athletics & Sports,Thai Restaurant,Fried Chicken Joint,Bank,NaN,NaN
6,M1K,-79.262029,6,Discount Store,Coffee Shop,Convenience Store,Hobby Shop,Department Store,Bus Station,NaN,NaN,NaN,NaN
7,M1L,-79.284577,6,Bakery,Bus Line,Park,Soccer Field,Bus Station,Metro Station,Intersection,Fast Food Restaurant,NaN,NaN
8,M1M,-79.239476,6,Motel,American Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,M1N,-79.264848,6,College Stadium,Skating Rink,Café,General Entertainment,NaN,NaN,NaN,NaN,NaN,NaN
10,M1P,-79.273304,6,Indian Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,NaN,NaN,NaN,NaN,NaN,NaN
11,M1R,-79.295849,6,Breakfast Spot,Smoke Shop,Bakery,Middle Eastern Restaurant,NaN,NaN,NaN,NaN,NaN,NaN
12,M1S,-79.262029,6,Breakfast Spot,Clothing Store,Lounge,Skating Rink,Latin American Restaurant,NaN,NaN,NaN,NaN,NaN
16,M2H,-79.363452,6,Fast Food Restaurant,Pool,Dog Run,Golf Course,Mediterranean Restaurant,NaN,NaN,NaN,NaN,NaN


This cluster can be named the 'others' clusters I supposed. Majority of the area in this cluster share some characteristics (i) have many venue categories, (ii) have a coffee shop.  

#### Cluster 8

In [80]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,M9W,-79.594054,7,Rental Car Location,Drugstore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cluster 9

In [81]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,-79.160497,8,Bar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cluster 10

In [82]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,M5N,-79.416936,9,Garden,Music Venue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cluster 11

In [83]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 10, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,M3K,-79.464763,10,Park,Airport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cluster 12

In [84]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 11, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M2L,-79.374714,11,Cafeteria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Cluster 6 and cluster 8-12 share similarities that they have only 1-2 venue categories.  These clusters can be categorised as outliers as their blend of venue categories are unique yet rare, with only one postal code assigned to each cluster.    